In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine, types

# Establish a connection to the SQL Server database
server = 'server'  # Replace 'your_server_name' with your SQL Server instance name
database = 'database'  # Replace 'your_database_name' with your database name
username = 'username'  # Replace 'your_username' with your SQL Server username
password = 'password'  # Replace 'your_password' with your SQL Server password
driver = 'ODBC Driver 17 for SQL Server'  # Replace with your SQL Server driver (e.g., 'ODBC+Driver+17+for+SQL+Server')

In [7]:
api_key = 'api_key'
endpoint = 'http://api.aviationstack.com/v1/flights'

params = {
    'access_key': api_key,
    'limit': 100,  # Limit the result to 100 flights per request
}

all_flights = []
page = 1

while True:
    # Update the page number in params
    params['offset'] = (page - 1) * params['limit']
    
    # Making the API request
    response = requests.get(endpoint, params=params)
    
    # Checking if the request was successful
    if response.status_code == 200:
        # Parsing the JSON response
        data = response.json()

        # Extracting flight data
        flights = data.get('data', [])
        
        if flights:
            # Add the current page of flights to the list
            all_flights.extend(flights)
            
            # If fewer flights were returned than the limit, stop paging
            if len(flights) < params['limit']:
                break
        else:
            break  # No more flight data available, exit the loop
    else:
        print(f"Error: {response.status_code}, {response.text}")
        break
    
    # Move to the next page
    page += 1

# Converting the list of all flights to a DataFrame
df = pd.DataFrame(all_flights)

# Saving the DataFrame to a CSV file
df.to_csv('C:\\Users\\cihat\\Downloads\\flights_data.csv', index=False)
print("Data saved to 'flights_data.csv'")

Error: 429, {"error":{"code":"usage_limit_reached","message":"Your monthly usage limit has been reached. Please upgrade your Subscription Plan."}}
Data saved to 'flights_data.csv'


In [2]:
flight_data = pd.read_csv('C:\\Users\\cihat\\Downloads\\flights_data.csv')

flight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10100 entries, 0 to 10099
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   flight_date    10100 non-null  object
 1   flight_status  10100 non-null  object
 2   departure      10100 non-null  object
 3   arrival        10100 non-null  object
 4   airline        10100 non-null  object
 5   flight         10100 non-null  object
 6   aircraft       19 non-null     object
 7   live           11 non-null     object
dtypes: object(8)
memory usage: 631.4+ KB


In [3]:
import ast

# Extracting the departure airport and assigning it to a new column
flight_data['departure_airport'] = flight_data['departure'].apply(
    lambda i: 'Unknown' if i is None else ast.literal_eval(i).get('airport', 'Unknown')
)

# Extracting the arrival airport and assigning it to a new column
flight_data['arrival_airport'] = flight_data['arrival'].apply(
    lambda i: 'Unknown' if i is None else ast.literal_eval(i).get('airport', 'Unknown')
)

In [4]:
flight_data['departure_airport'][2] = flight_data['departure_airport'][2].replace('\r\n', '')

#stahp

AttributeError: 'NoneType' object has no attribute 'replace'

In [5]:
flight_data['departure_timezone'] = flight_data['departure'].apply(
    lambda i: 'Unknown' if i is None else ast.literal_eval(i).get('timezone', 'Unknown')
)

flight_data['arrival_timezone'] = flight_data['arrival'].apply(
    lambda i: 'Unknown' if i is None else ast.literal_eval(i).get('timezone', 'Unknown')
)

In [6]:
flight_data['airline_name'] = flight_data['airline'].apply(
    lambda i: 'Unknown' if i is None else ast.literal_eval(i).get('name', 'Unknown')
)

flight_data['flight_number'] = flight_data['flight'].apply(
    lambda i: 'Unknown' if i is None else ast.literal_eval(i).get('number', 'Unknown')
)

In [7]:
flight_data['departure_timezone'] = flight_data['departure_timezone'].fillna('Unknown/Unknown')
flight_data['arrival_timezone'] = flight_data['arrival_timezone'].fillna('Unknown/Unknown')

In [8]:
flight_data

,flight_date,flight_status,departure,arrival,airline,flight,aircraft,live,departure_airport,arrival_airport,departure_timezone,arrival_timezone,airline_name,flight_number
0,2024-09-10,scheduled,{'airport': 'Chengdu Tianfu International Airp...,"{'airport': None, 'timezone': None, 'iata': 'J...","{'name': 'Air China LTD', 'iata': 'CA', 'icao'...","{'number': '2553', 'iata': 'CA2553', 'icao': '...",NaN,NaN,Chengdu Tianfu International Airport,None,Asia/Chongqing,Unknown/Unknown,Air China LTD,2553
1,2024-09-10,scheduled,{'airport': 'Chengdu Tianfu International Airp...,"{'airport': None, 'timezone': None, 'iata': 'J...","{'name': 'Shenzhen Airlines', 'iata': 'ZH', 'i...","{'number': '4043', 'iata': 'ZH4043', 'icao': '...",NaN,NaN,Chengdu Tianfu International Airport,None,Asia/Chongqing,Unknown/Unknown,Shenzhen Airlines,4043
2,2024-09-10,scheduled,"{'airport': None, 'timezone': None, 'iata': 'J...","{'airport': 'Changsha', 'timezone': 'Asia/Shan...","{'name': 'Kunming Airlines', 'iata': 'KY', 'ic...","{'number': '3125', 'iata': 'KY3125', 'icao': '...",NaN,NaN,None,Changsha,Unknown/Unknown,Asia/Shanghai,Kunming Airlines,3125
3,2024-09-10,scheduled,"{'airport': None, 'timezone': None, 'iata': 'J...","{'airport': 'Changsha', 'timezone': 'Asia/Shan...","{'name': 'Shenzhen Airlines', 'iata': 'ZH', 'i...","{'number': '3125', 'iata': 'ZH3125', 'icao': '...",NaN,NaN,None,Changsha,Unknown/Unknown,Asia/Shanghai,Shenzhen Airlines,3125
4,2024-09-10,scheduled,"{'airport': None, 'timezone': None, 'iata': 'J...","{'airport': 'Changsha', 'timezone': 'Asia/Shan...","{'name': 'Tibet Airlines', 'iata': 'TV', 'icao...","{'number': '8125', 'iata': 'TV8125', 'icao': '...",NaN,NaN,None,Changsha,Unknown/Unknown,Asia/Shanghai,Tibet Airlines,8125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,2024-09-10,scheduled,"{'airport': 'Qingdao', 'timezone': 'Asia/Shang...","{'airport': 'Beijing Capital International', '...","{'name': 'Shandong Airlines', 'iata': 'SC', 'i...","{'number': '4641', 'iata': 'SC4641', 'icao': '...",NaN,NaN,Qingdao,Beijing Capital International,Asia/Shanghai,Asia/Shanghai,Shandong Airlines,4641
10096,2024-09-10,scheduled,"{'airport': 'Qingdao', 'timezone': 'Asia/Shang...","{'airport': 'Beijing Capital International', '...","{'name': 'Shenzhen Airlines', 'iata': 'ZH', 'i...","{'number': '2633', 'iata': 'ZH2633', 'icao': '...",NaN,NaN,Qingdao,Beijing Capital International,Asia/Shanghai,Asia/Shanghai,Shenzhen Airlines,2633
10097,2024-09-10,scheduled,"{'airport': 'Qingdao', 'timezone': 'Asia/Shang...","{'airport': 'Beijing Capital International', '...","{'name': 'Tibet Airlines', 'iata': 'TV', 'icao...","{'number': '3143', 'iata': 'TV3143', 'icao': '...",NaN,NaN,Qingdao,Beijing Capital International,Asia/Shanghai,Asia/Shanghai,Tibet Airlines,3143
10098,2024-09-10,scheduled,"{'airport': 'Qingdao', 'timezone': 'Asia/Shang...","{'airport': 'Beijing Capital International', '...","{'name': 'China Express Air', 'iata': 'G5', 'i...","{'number': '6479', 'iata': 'G56479', 'icao': '...",NaN,NaN,Qingdao,Beijing Capital International,Asia/Shanghai,Asia/Shanghai,China Express Air,6479


In [9]:
flight_data[['departure_continent', 'departure_city']] = flight_data['departure_timezone'].str.split('/', expand=True, n=1)

flight_data[['arrival_continent', 'arrival_city']] = flight_data['arrival_timezone'].str.split('/', expand=True, n=1)

In [10]:
flight_data['departure_continent'].unique()
flight_data['departure_city'].unique()

array(['Chongqing', 'Unknown', 'Kuala_Lumpur', 'Tokyo', 'Shanghai',
       'Berlin', 'Manila', 'Seoul', 'Jayapura', 'Yekaterinburg',
       'Nairobi', 'Dar_es_Salaam', 'Moscow', 'Kolkata', 'Tirane',
       'Ho_Chi_Minh', 'Tehran', 'Madrid', 'Dhaka', 'Algiers', 'Jakarta',
       'Makassar', 'Rangoon', 'Karachi', 'Cairo', 'Brisbane', 'Melbourne',
       'Sydney', 'Auckland', 'Tallinn', 'Muscat', 'Baghdad', 'Bamako',
       'Singapore', 'Riyadh', 'Bangkok', 'Istanbul', 'Vladivostok',
       'Kuching', 'Bucharest', 'Macau', 'Qatar', 'Taipei', 'Novosibirsk',
       'Baku', 'Bishkek', 'Antananarivo', 'Kigali', 'Perth', 'Kathmandu',
       'Aqtau', 'Tunis', 'Dili', 'Krasnoyarsk', 'Hobart', 'Hong_Kong',
       None, 'Guadalcanal', 'Currie', 'Sofia', 'Dublin', 'Kampala',
       'Dushanbe', 'Dubai', 'Port_Moresby', 'Warsaw', 'Tashkent',
       'Yakutsk', 'Adelaide', 'Chisinau', 'Rome', 'Fiji', 'Maldives',
       'Bahrain', 'Vilnius', 'London', 'Samara', 'Nicosia', 'Pontianak',
       'Phnom_Penh

In [11]:
flight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10100 entries, 0 to 10099
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   flight_date          10100 non-null  object
 1   flight_status        10100 non-null  object
 2   departure            10100 non-null  object
 3   arrival              10100 non-null  object
 4   airline              10100 non-null  object
 5   flight               10100 non-null  object
 6   aircraft             19 non-null     object
 7   live                 11 non-null     object
 8   departure_airport    9992 non-null   object
 9   arrival_airport      9959 non-null   object
 10  departure_timezone   10100 non-null  object
 11  arrival_timezone     10100 non-null  object
 12  airline_name         9929 non-null   object
 13  flight_number        10027 non-null  object
 14  departure_continent  10100 non-null  object
 15  departure_city       9969 non-null   object
 16  arri

In [12]:
flight_data.loc[flight_data['departure_continent'] == '+8', 'departure_continent'] = 'Asia'
flight_data.loc[flight_data['arrival_continent'] == '+8', 'departure_continent'] = 'Asia'

In [13]:
flight_data['departure_delay'] = flight_data['departure'].apply(
    lambda i: 'Unknown' if i is None else ast.literal_eval(i).get('delay', 'Unknown')
)

flight_data['arrival_delay'] = flight_data['arrival'].apply(
    lambda i: 'Unknown' if i is None else ast.literal_eval(i).get('delay', 'Unknown')
)

In [14]:
flight_data.keys()

Index(['flight_date', 'flight_status', 'departure', 'arrival', 'airline',
       'flight', 'aircraft', 'live', 'departure_airport', 'arrival_airport',
       'departure_timezone', 'arrival_timezone', 'airline_name',
       'flight_number', 'departure_continent', 'departure_city',
       'arrival_continent', 'arrival_city', 'departure_delay',
       'arrival_delay'],
      dtype='object')

In [16]:
flight_data1 = flight_data.drop_duplicates()

flight_data1

,flight_date,flight_status,departure,arrival,airline,flight,aircraft,live,departure_airport,arrival_airport,departure_timezone,arrival_timezone,airline_name,flight_number,departure_continent,departure_city,arrival_continent,arrival_city,departure_delay,arrival_delay
0,2024-09-10,scheduled,{'airport': 'Chengdu Tianfu International Airp...,"{'airport': None, 'timezone': None, 'iata': 'J...","{'name': 'Air China LTD', 'iata': 'CA', 'icao'...","{'number': '2553', 'iata': 'CA2553', 'icao': '...",NaN,NaN,Chengdu Tianfu International Airport,None,Asia/Chongqing,Unknown/Unknown,Air China LTD,2553,Asia,Chongqing,Unknown,Unknown,NaN,NaN
1,2024-09-10,scheduled,{'airport': 'Chengdu Tianfu International Airp...,"{'airport': None, 'timezone': None, 'iata': 'J...","{'name': 'Shenzhen Airlines', 'iata': 'ZH', 'i...","{'number': '4043', 'iata': 'ZH4043', 'icao': '...",NaN,NaN,Chengdu Tianfu International Airport,None,Asia/Chongqing,Unknown/Unknown,Shenzhen Airlines,4043,Asia,Chongqing,Unknown,Unknown,NaN,NaN
2,2024-09-10,scheduled,"{'airport': None, 'timezone': None, 'iata': 'J...","{'airport': 'Changsha', 'timezone': 'Asia/Shan...","{'name': 'Kunming Airlines', 'iata': 'KY', 'ic...","{'number': '3125', 'iata': 'KY3125', 'icao': '...",NaN,NaN,None,Changsha,Unknown/Unknown,Asia/Shanghai,Kunming Airlines,3125,Unknown,Unknown,Asia,Shanghai,NaN,NaN
3,2024-09-10,scheduled,"{'airport': None, 'timezone': None, 'iata': 'J...","{'airport': 'Changsha', 'timezone': 'Asia/Shan...","{'name': 'Shenzhen Airlines', 'iata': 'ZH', 'i...","{'number': '3125', 'iata': 'ZH3125', 'icao': '...",NaN,NaN,None,Changsha,Unknown/Unknown,Asia/Shanghai,Shenzhen Airlines,3125,Unknown,Unknown,Asia,Shanghai,NaN,NaN
4,2024-09-10,scheduled,"{'airport': None, 'timezone': None, 'iata': 'J...","{'airport': 'Changsha', 'timezone': 'Asia/Shan...","{'name': 'Tibet Airlines', 'iata': 'TV', 'icao...","{'number': '8125', 'iata': 'TV8125', 'icao': '...",NaN,NaN,None,Changsha,Unknown/Unknown,Asia/Shanghai,Tibet Airlines,8125,Unknown,Unknown,Asia,Shanghai,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,2024-09-10,scheduled,"{'airport': 'Qingdao', 'timezone': 'Asia/Shang...","{'airport': 'Beijing Capital International', '...","{'name': 'Shandong Airlines', 'iata': 'SC', 'i...","{'number': '4641', 'iata': 'SC4641', 'icao': '...",NaN,NaN,Qingdao,Beijing Capital International,Asia/Shanghai,Asia/Shanghai,Shandong Airlines,4641,Asia,Shanghai,Asia,Shanghai,NaN,NaN
10096,2024-09-10,scheduled,"{'airport': 'Qingdao', 'timezone': 'Asia/Shang...","{'airport': 'Beijing Capital International', '...","{'name': 'Shenzhen Airlines', 'iata': 'ZH', 'i...","{'number': '2633', 'iata': 'ZH2633', 'icao': '...",NaN,NaN,Qingdao,Beijing Capital International,Asia/Shanghai,Asia/Shanghai,Shenzhen Airlines,2633,Asia,Shanghai,Asia,Shanghai,NaN,NaN
10097,2024-09-10,scheduled,"{'airport': 'Qingdao', 'timezone': 'Asia/Shang...","{'airport': 'Beijing Capital International', '...","{'name': 'Tibet Airlines', 'iata': 'TV', 'icao...","{'number': '3143', 'iata': 'TV3143', 'icao': '...",NaN,NaN,Qingdao,Beijing Capital International,Asia/Shanghai,Asia/Shanghai,Tibet Airlines,3143,Asia,Shanghai,Asia,Shanghai,NaN,NaN
10098,2024-09-10,scheduled,"{'airport': 'Qingdao', 'timezone': 'Asia/Shang...","{'airport': 'Beijing Capital International', '...","{'name': 'China Express Air', 'iata': 'G5', 'i...","{'number': '6479', 'iata': 'G56479', 'icao': '...",NaN,NaN,Qingdao,Beijing Capital International,Asia/Shanghai,Asia/Shanghai,China Express Air,6479,Asia,Shanghai,Asia,Shanghai,NaN,NaN


In [15]:
from pandas_gbq import to_gbq
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\cihat\\Downloads\\new-one-420600-8e7e0e7eead4.json"


project_id = 'new-one-420600'
dataset_id = 'trying_gcp'
table_name = 'flight_dataset'


# Define the full table name in the format dataset_id.table_id
table_id = f'{dataset_id}.{table_name}'

# Upload the DataFrame to BigQuery
to_gbq(flight_data, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [17]:
# Define the connection string
conn_str = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}'

# Create a SQLAlchemy engine
engine = create_engine(conn_str)


# Specify the table name
table_name = 'Flight_Data'  # Replace 'YourTableName' with your table name


flight_data.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Close the SQLAlchemy engine
engine.dispose()


print("Data has been successfully uploaded to the SQL Server database.")

Data has been successfully uploaded to the SQL Server database.


In [22]:
corr = flight_data[['departure_delay', 'arrival_delay']].corr()

print(corr)

                 departure_delay  arrival_delay
departure_delay         1.000000       0.885237
arrival_delay           0.885237       1.000000
